this code uses some progarms direactly from the os that might need to be installed on the operators machine 

---

so far we have used 
dcm2niix to convert dcm to nii files we do this for proformance reasons as the data is lare and saving time is nacceary 




In [1]:
import subprocess

def convert_dicom_to_nifti(dicom_dir, output_dir):
    command = ['dcm2niix', '-o', output_dir, dicom_dir]
    subprocess.run(command, check=True)

dicom_directory = "./DATA/RAW/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623/"
output_path = "./DATA/testing/I238623"

print("starting")

convert_dicom_to_nifti(dicom_directory, output_path)

print("done")

starting
Chris Rorden's dcm2niiX version v1.0.20220720  (JP2:OpenJPEG) GCC13.2.0 x86-64 (64-bit Linux)
Found 6720 DICOM file(s)
Philips Scaling Values RS:RI:SS = 1.22222:0:0.000600573 (see PMC3998685)
Convert 6720 DICOM as ./DATA/testing/I238623/I238623_Resting_State_fMRI_20110602075636_501a (64x64x48x140)
Conversion required 3.497350 seconds (3.497098 for core code).
done


In [1]:
# sanity check to see if nii coversion was sucssuful 
# NOTE this looks fine for now need to check with ANDI documentation 
# to see if this is realy the size of the data 

import nibabel as nib

def check_nii_dimensions(nii_file_path):
    nii_image = nib.load(nii_file_path)
    dimensions = nii_image.shape  # Get the dimensions of the NIfTI file
    print(f"NIfTI Image Dimensions: {dimensions}")

# Example usage
nii_file_path = "./DATA/testing/I238623/I238623_Resting_State_fMRI_20110602075636_501.nii"

print("starting ")
check_nii_dimensions(nii_file_path)

print("fibished")

starting 
NIfTI Image Dimensions: (64, 64, 48, 140)
fibished


In [ ]:
# sanity check continues 
# NOTE this looks fine for now need to check with ANDI documentation 
# to see if this is realy the size of the data 

import os

def count_dicom_files(dicom_dir):
    dicom_files = [f for f in os.listdir(dicom_dir) if f.lower().endswith('.dcm')]
    return len(dicom_files)

# Count DICOM files
dicom_count = count_dicom_files("./DATA/RAW/002_S_0295/Resting_State_fMRI/2011-06-02_07_56_36.0/I238623/")
print(f"Number of DICOM files: {dicom_count}")

In [ ]:
#  converting all dicom files in RAW data to nii files 
# TODO find a way to stop printing so much stuff  

def convert_all_dicoms(base_dir, output_base_dir):
    
    for curr_path, dirs, files in os.walk(base_dir):
        
        # Check if we are in the deepest folder containing DICOM files
        if any(file.lower().endswith('.dcm') for file in files):
            
            relative_path = os.path.relpath(curr_path, base_dir)
            output_dir = os.path.join(output_base_dir, relative_path)
            os.makedirs(output_dir, exist_ok=True)
            
            # Convert using dcm2niix
            command = ['dcm2niix', '-o', output_dir, curr_path]
            subprocess.run(command, check=True)
            print(f"Converted DICOM folder: {curr_path} to {output_dir}")

base_dicom_dir = "./DATA/RAW/"
output_path = "./DATA/testing/"

print("starting")

convert_all_dicoms(base_dicom_dir,output_path)

print("end")


In [ ]:

from nipype import Node
from nipype.interfaces.fsl import MCFLIRT
from nipype import Workflow, Node

def perform_realignment(input_file, output_file):
    realign_node = Node(MCFLIRT(), name='realignment')
    realign_node.inputs.in_file = input_file
    realign_node.inputs.out_file = output_file
    realign_node.inputs.cost = "normcorr"  # Example parameter, you can change this
    realign_node.inputs.save_mats = True
    realign_node.inputs.save_plots = True
    return realign_node

# Usage
realignment_node = perform_realignment(nii_file_path, 'realigned_data.nii')
wf_realign = Workflow(name='realignment_workflow')
wf_realign.add_nodes([realign_node])
wf_realign.run()




In [2]:
import fslpy
import fsl.wrappers as fslw

def perform_realignment(input_file, output_file):
    # Use MCFLIRT for motion correction
    fslw.mcflirt(infile=input_file, outfile=output_file, save_mats=True, save_plots=True, cost='normcorr')

input_file = 'path/to/your/input_file.nii.gz'
output_file = 'path/to/your/output_file.nii.gz'

perform_realignment(input_file, output_file)





ModuleNotFoundError: No module named 'fslpy'

In [ ]:
import subprocess
subprocess.run(['fslmaths', '--help'])

In [ ]:
# view ini data 

fmri_img = nib.load('your_fmri_data.nii')
fmri_data = fmri_img.get_fdata()

# Get the shape of the data
print("fMRI data shape:", fmri_data.shape)
